In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 4ba1145a-92e9-4073-ae50-d3c791b0145c
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 4ba1145a-92e9-4073-ae50-d3c791b0145c to get into ready status...
Session 4ba1145a-92e9-4073-ae50-d3c791b0145c ha

In [34]:
from pyspark.sql.functions import *
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

In [8]:
s3_path = "s3://spotify-etl-pipeline-prudhvi/raw_data/to_processed/"
source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    format="json",
    connection_options={"paths": [s3_path]}
)

In [9]:
spotify_df = source_dyf.toDF()

In [14]:
# spotify_df.show()

+--------------------+--------------------+-----+----+------+--------+-----+
|                href|               items|limit|next|offset|previous|total|
+--------------------+--------------------+-----+----+------+--------+-----+
|https://api.spoti...|[{2015-01-15T12:3...|  100|NULL|     0|    NULL|    5|
+--------------------+--------------------+-----+----+------+--------+-----+


In [10]:
# df = spotify_df

In [15]:
# df.withColumn("items", explode("items")).select(
#     col("items.track.album.id").alias("album_id"),
#     col("items.track.album.name").alias("album_name"),
#     col("items.track.album.release_date").alias("release_date"),
#     col("items.track.album.total_tracks").alias("total_tracks"),
#     col("items.track.album.external_urls.spotify").alias("url")
# ).drop_duplicates(['album_id']).show()

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|0ivM6kSawaug0j3tZ...|All The Best (Spa...|  2007-01-01|          18|https://open.spot...|
|2pANdqPvxInB0YvcD...|Progressive Psy T...|  2012-04-02|          20|https://open.spot...|
|2usKFntxa98WHMcyW...|Glenn Horiuchi Tr...|  2011-04-01|           8|https://open.spot...|
|4hnqM0JK4CM1phwfq...|   This Is Happening|  2010-05-17|           9|https://open.spot...|
|6nlfkk5GoXRL1nktl...|Wellness & Dreami...|  2015-01-09|          25|https://open.spot...|
+--------------------+--------------------+------------+------------+--------------------+


In [19]:
# df_artist_exploded = df.select(explode(col("items")).alias("items")).select(explode(col("items.track.artists")).alias("artists"))

In [25]:
# df_artist_exploded.select(
#     col("artists.id").alias("artist_id"),
#     col("artists.name").alias("artist_name"),
#     col("artists.external_urls.spotify").alias("url")
# ).drop_duplicates(['artist_id']).show()

+--------------------+-------------------+--------------------+
|           artist_id|        artist_name|                 url|
+--------------------+-------------------+--------------------+
|066X20Nz7iquqkkCW...|    LCD Soundsystem|https://open.spot...|
|272ArH9SUAlslQqsS...|Glenn Horiuchi Trio|https://open.spot...|
|2KftmGt9sk1yLjsAo...|           Zucchero|https://open.spot...|
|5VQE4WOzPu9h3HnGL...|       Vlasta Marek|https://open.spot...|
|6eSdhw46riw2OUHgM...|             Odiseo|https://open.spot...|
+--------------------+-------------------+--------------------+


In [26]:
# df_explode = df.select(explode(col("items")).alias("item"))

In [28]:
# df_explode.select(
#     col("item.track.id").alias("song_id"),
#     col("item.track.name").alias("song_name"),
#     col("item.track.duration_ms").alias("duration_ms"),
#     col("item.track.external_urls.spotify").alias("url"),
#     col("item.track.popularity").alias("popularity"),
#     col("item.added_at").alias("song_added"),
#     col("item.track.album.id").alias("album_id"),
#     col("item.track.artists")[0]["id"].alias("artist_id")
# ).drop_duplicates(['song_id']).show()

+--------------------+--------------------+-----------+--------------------+----------+--------------------+--------------------+--------------------+
|             song_id|           song_name|duration_ms|                 url|popularity|          song_added|            album_id|           artist_id|
+--------------------+--------------------+-----------+--------------------+----------+--------------------+--------------------+--------------------+
|2E2znCPaS8anQe21G...|You Are So Beautiful|     176093|https://open.spot...|         0|2015-01-15T12:41:10Z|0ivM6kSawaug0j3tZ...|2KftmGt9sk1yLjsAo...|
|4Cy0NHJ8Gh0xMdwyM...|          All I Want|     401440|https://open.spot...|        48|2015-01-15T12:22:30Z|4hnqM0JK4CM1phwfq...|066X20Nz7iquqkkCW...|
|4rzfv0JLZfVhOhbSQ...|                 Api|     376000|https://open.spot...|         1|2015-01-15T12:39:22Z|2pANdqPvxInB0YvcD...|6eSdhw46riw2OUHgM...|
|5o3jMYOSbaVz3tkgw...|                  Is|     730066|https://open.spot...|         0|2015-01

In [29]:
def process_albums(df):
    df = df.withColumn("items", explode("items")).select(
        col("items.track.album.id").alias("album_id"),
        col("items.track.album.name").alias("album_name"),
        col("items.track.album.release_date").alias("release_date"),
        col("items.track.album.total_tracks").alias("total_tracks"),
        col("items.track.album.external_urls.spotify").alias("url")
    ).drop_duplicates(['album_id'])
    return df

def process_artists(df):
    df_items_exploded = df.select(explode(col("items")).alias("item"))
    
    df_artists_exploded = df_items_exploded.select(explode(col("item.track.artists")).alias("artist"))
    
    df_artists = df_artists_exploded.select(
        col("artist.id").alias("artist_id"),
        col("artist.name").alias("artist_name"),
        col("artist.external_urls.spotify").alias("url")
    ).drop_duplicates(['artist_id'])
    
    return df_artists

def process_songs(df):
    df_exploded = df.select(explode(col("items")).alias("item"))
    
    df_songs = df_exploded.select(
        col("item.track.id").alias("song_id"),
        col("item.track.name").alias("song_name"),
        col("item.track.duration_ms").alias("duration_ms"),
        col("item.track.external_urls.spotify").alias("url"),
        col("item.track.popularity").alias("popularity"),
        col("item.added_at").alias("song_added"),
        col("item.track.album.id").alias("album_id"),
        col("item.track.artists")[0]["id"].alias("artist_id")
    ).drop_duplicates(['song_id'])
    
    df_songs = df_songs.withColumn("song_added", to_date(col("song_added")))
    
    return df_songs

In [30]:
album_df = process_albums(spotify_df)
album_df.show()

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|0ivM6kSawaug0j3tZ...|All The Best (Spa...|  2007-01-01|          18|https://open.spot...|
|2pANdqPvxInB0YvcD...|Progressive Psy T...|  2012-04-02|          20|https://open.spot...|
|2usKFntxa98WHMcyW...|Glenn Horiuchi Tr...|  2011-04-01|           8|https://open.spot...|
|4hnqM0JK4CM1phwfq...|   This Is Happening|  2010-05-17|           9|https://open.spot...|
|6nlfkk5GoXRL1nktl...|Wellness & Dreami...|  2015-01-09|          25|https://open.spot...|
+--------------------+--------------------+------------+------------+--------------------+


In [31]:
artist_df = process_artists(spotify_df)
artist_df.show()

+--------------------+-------------------+--------------------+
|           artist_id|        artist_name|                 url|
+--------------------+-------------------+--------------------+
|066X20Nz7iquqkkCW...|    LCD Soundsystem|https://open.spot...|
|272ArH9SUAlslQqsS...|Glenn Horiuchi Trio|https://open.spot...|
|2KftmGt9sk1yLjsAo...|           Zucchero|https://open.spot...|
|5VQE4WOzPu9h3HnGL...|       Vlasta Marek|https://open.spot...|
|6eSdhw46riw2OUHgM...|             Odiseo|https://open.spot...|
+--------------------+-------------------+--------------------+


In [32]:
songs_df = process_songs(spotify_df)
songs_df.show()

+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|             song_id|           song_name|duration_ms|                 url|popularity|song_added|            album_id|           artist_id|
+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|2E2znCPaS8anQe21G...|You Are So Beautiful|     176093|https://open.spot...|         0|2015-01-15|0ivM6kSawaug0j3tZ...|2KftmGt9sk1yLjsAo...|
|4Cy0NHJ8Gh0xMdwyM...|          All I Want|     401440|https://open.spot...|        48|2015-01-15|4hnqM0JK4CM1phwfq...|066X20Nz7iquqkkCW...|
|4rzfv0JLZfVhOhbSQ...|                 Api|     376000|https://open.spot...|         1|2015-01-15|2pANdqPvxInB0YvcD...|6eSdhw46riw2OUHgM...|
|5o3jMYOSbaVz3tkgw...|                  Is|     730066|https://open.spot...|         0|2015-01-15|6nlfkk5GoXRL1nktl...|5VQE4WOzPu9h3HnGL...|
|6hvFrZNocdt2

In [38]:
def write_to_s3(df, path_suffix, format_type = "csv"):
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    
    glueContext.write_dynamic_frame.from_options(
        frame = dynamic_frame,
        connection_type = "s3",
        connection_options = {"path": f"s3://spotify-etl-pipeline-prudhvi/transformed_data/{path_suffix}/"},
        format = format_type
    )

In [39]:
write_to_s3(album_df, "album_data/album_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), 'csv')

In [41]:
write_to_s3(artist_df, "artist_data/artist_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), 'csv')

In [43]:
write_to_s3(songs_df, "songs_data/songs_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), 'csv')